1. IMPORT LIBRARY

In [38]:
from datetime import datetime, timedelta, date
import pandas as pd
import pyodbc 


2. CONNECT TO SQL SERVER

In [39]:
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-MV8RPS5;"
    "DATABASE=Data_Python;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)

In [40]:
df_associated_rule = pd.read_sql_query(''' 
SELECT *
  FROM [Data_Python].[dbo].[Groceries_dataset]
    ; ''', conn)
df_associated_rule

C:\Users\Tran Minh Huy\AppData\Local\Temp\ipykernel_22324\3331954196.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_associated_rule = pd.read_sql_query('''


,Member_number,Date,itemDescription,Day,Season,Vacation
0,100000000,2015-07-21,tropical fruit,Sun,Winter,True
1,2552,2015-01-05,whole milk,Mon,Summer,False
2,2300,2015-09-19,pip fruit,Tue,Summer,False
3,-1000000,2015-12-12,other vegetables,Wed,Summer,False
4,3037,2015-02-01,whole milk,Thu,Summer,False
...,...,...,...,...,...,...
38779,4471,2014-10-08,sliced cheese,Sat,Winter,True
38780,2022,2014-02-23,candy,Sun,Winter,True
38781,1097,2014-04-16,cake bar,Mon,Winter,True
38782,1510,2014-12-03,fruit/vegetable juice,Tue,Winter,True


3. CLEARNING DATA

3.1 REMOVE DUPLIATES

In [41]:
df_associated_rule = df_associated_rule.drop_duplicates()
df_associated_rule.shape

(38448, 6)

3.2 Detec Outlier Or Not Interquartile Range (IQR) Method

In [42]:
# Calculate Q1, Q3, and IQR
Q1 = df_associated_rule['Member_number'].quantile(0.25)
Q3 = df_associated_rule['Member_number'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers_iqr = df_associated_rule[(df_associated_rule['Member_number'] < lower_bound) | (df_associated_rule['Member_number'] > upper_bound)]
print("Outliers using IQR method:")
print(outliers_iqr)

# Remove outliers
df_associated_rule = df_associated_rule[(df_associated_rule['Member_number'] >= lower_bound) & (df_associated_rule['Member_number'] <= upper_bound)]
print("\nDataFrame after removing outliers (IQR method):")
print(df_associated_rule.shape)

Outliers using IQR method:
   Member_number        Date   itemDescription  Day  Season  Vacation
0      100000000  2015-07-21    tropical fruit  Sun  Winter      True
3       -1000000  2015-12-12  other vegetables  Wed  Summer     False

DataFrame after removing outliers (IQR method):
(38446, 6)


3.3 Remove Irrevalanet Data

In [43]:
df_associated_rule = df_associated_rule[['Member_number','Date','itemDescription']]
df_associated_rule

,Member_number,Date,itemDescription
1,2552,2015-01-05,whole milk
2,2300,2015-09-19,pip fruit
4,3037,2015-02-01,whole milk
5,4941,2015-02-14,rolls/buns
6,4501,2015-05-08,other vegetables
...,...,...,...
38779,4471,2014-10-08,sliced cheese
38780,2022,2014-02-23,candy
38781,1097,2014-04-16,cake bar
38782,1510,2014-12-03,fruit/vegetable juice


3.4 SO ON YOU NAME IT

In [44]:
df_associated_rule.rename(columns={'Member_number':'member_number', 'Date':'date','itemDescription':'item_description'},inplace=True)

4. PUSH TO SQL SEVER DATA CLEAR

In [46]:
df_associated_rule.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38446 entries, 1 to 38783
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   member_number     38446 non-null  int64 
 1   date              38425 non-null  object
 2   item_description  38425 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [47]:
df_associated_rule.dropna(inplace=True)

In [48]:
df_associated_rule.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38425 entries, 1 to 38783
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   member_number     38425 non-null  int64 
 1   date              38425 non-null  object
 2   item_description  38425 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [49]:
import pyodbc

# --- Connect ---
conn = pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-MV8RPS5;"
    "DATABASE=Data_Python;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)
cursor = conn.cursor()

# --- Map pandas dtype -> SQL Server type ---
dtype_map = lambda dt: "INT" if pd.api.types.is_integer_dtype(dt) else \
                       "FLOAT" if pd.api.types.is_float_dtype(dt) else \
                       "BIT" if pd.api.types.is_bool_dtype(dt) else \
                       "DATETIME" if pd.api.types.is_datetime64_any_dtype(dt) else \
                       "VARCHAR(255)"

table_name = "data_Associate_Rule_Insights"

# --- Drop + Create table ---
cursor.execute(f"IF OBJECT_ID('dbo.{table_name}', 'U') IS NOT NULL DROP TABLE dbo.{table_name}")
cols = ", ".join([f"[{c}] {dtype_map(df_associated_rule[c].dtype)}" for c in df_associated_rule.columns])
cursor.execute(f"CREATE TABLE dbo.{table_name} ({cols})")

# --- Insert rows ---
cursor.fast_executemany = True
cursor.executemany(f"INSERT INTO dbo.{table_name} VALUES ({','.join(['?']*len(df_associated_rule.columns))})", df_associated_rule.values.tolist())
conn.commit()